# Interfaces

Classes are objects. Type and class can mostly be used to mean the same in Python but there are differences in attributes between the two.

In [44]:
class MySimpleInterface:
    """Simple interface but with no enforcement, simply using standard subclassing."""

    def print_message(self, message: str) -> None:
        """Print message method with no implementation."""
        pass

    def my_other_interface_function(self):
        """There is no requirement with simple subclassing to fully implement this interface. Will work without it."""
        pass        # is this an interface in the true sense of the word?

class MyConcreteClass(MySimpleInterface):           # does not fully implement the interface

    def print_message(self, message: str) -> None:
        print("MyConcreteClass message:", message)

class MyOtherConcreteClass(MySimpleInterface):      # fully implements the interface

    def print_message(self, message: str) -> None:
        print("MyOtherConcreteClass message:", message)

    def my_other_interface_function(self):
        print("MyOtherConcreteClass message in my_other_interface_function!")

We can pass interfaces to functions to use

In [45]:
def some_function(my_interface: MySimpleInterface) -> None:
    print("some_function: my_interface =", str(my_interface)[1:-1])
    my_interface.print_message("Some random message!")
    # even though we have not fully implemented the interface we can however call my_other_interface
    my_interface.my_other_interface_function()

my_concrete_class = MyConcreteClass()
my_other_concrete_class = MyOtherConcreteClass()

some_function(my_concrete_class)
some_function(my_other_concrete_class)

some_function: my_interface = __main__.MyConcreteClass object at 0x000001846CC41C30
MyConcreteClass message: Some random message!
some_function: my_interface = __main__.MyOtherConcreteClass object at 0x000001846CC416F0
MyOtherConcreteClass message: Some random message!
MyOtherConcreteClass message in my_other_interface_function!


We could override using a `metaclass` to enforce type checking. See https://realpython/python-interface

In [46]:
class SimpleMeta(type):                                 # metaclass describes how a class should work

    def __instancecheck__(self, instance):
        print("SimpleMeta: __instancecheck__")
        return self.__subclasscheck__(type(instance))

    def __subclasscheck__(self, subclass):
        print("SimpleMeta: __subclasscheck__")
        print("SimpleMeta: self =", str(self)[1:-1])
        print("SimpleMeta: subclass =", str(subclass)[1:-1])
        # see if subclass actually implements the interface
        return hasattr(subclass, 'print_message') and callable(subclass.print_message) and \
            hasattr(subclass, 'my_other_interface_function') and callable(subclass.my_other_interface_function)

# this is a virtual base class of MyFullImplementedConcreteClass
# below as it implements the interface defined in metaclass
class MySimpleInterface(metaclass=SimpleMeta):
    """__subclasscheck__ implicitly makes print_message and my_other_interface available"""
    pass

class MyConcreteClass:
    def print_message(self, message: str) -> None:
        print("MyConcreteClass: message =", message)

# note implicit super-classing, not MyFullImplementedConcreteClass(MySimpleInterface)
# which is also perfectly fine to do so appears in MRO
class MyFullImplementedConcreteClass:
    def print_message(self, message: str) -> None:
        print("MyOtherConcreteClass: message =", message)

    def my_other_interface_function(self):
        print("MyFullImplementedConcreteClass: in my_other_interface_function!")

Using a metaclass we can see if a class is really of type `MySimpleInterface`

In [47]:
print("issubclass(MyConcreteClass, MySimpleInterface) =", issubclass(MyConcreteClass, MySimpleInterface))
print("issubclass(MyFullImplementedConcreteClass, MySimpleInterface) =",
      issubclass(MyFullImplementedConcreteClass, MySimpleInterface))

SimpleMeta: __subclasscheck__
SimpleMeta: self = class '__main__.MySimpleInterface'
SimpleMeta: subclass = class '__main__.MyConcreteClass'
issubclass(MyConcreteClass, MySimpleInterface) = False
SimpleMeta: __subclasscheck__
SimpleMeta: self = class '__main__.MySimpleInterface'
SimpleMeta: subclass = class '__main__.MyFullImplementedConcreteClass'
issubclass(MyFullImplementedConcreteClass, MySimpleInterface) = True


Now let's create some instances

In [48]:
my_full_concrete_class = MyFullImplementedConcreteClass()
my_concrete_class = MyConcreteClass()
print("isinstance(my_full_concrete_class, MySimpleInterface) =", isinstance(my_full_concrete_class, MySimpleInterface))
print("isinstance(my_concrete_class, MySimpleInterface) =", isinstance(my_concrete_class, MySimpleInterface))

SimpleMeta: __instancecheck__
SimpleMeta: __subclasscheck__
SimpleMeta: self = class '__main__.MySimpleInterface'
SimpleMeta: subclass = class '__main__.MyFullImplementedConcreteClass'
isinstance(my_full_concrete_class, MySimpleInterface) = True
SimpleMeta: __instancecheck__
SimpleMeta: __subclasscheck__
SimpleMeta: self = class '__main__.MySimpleInterface'
SimpleMeta: subclass = class '__main__.MyConcreteClass'
isinstance(my_concrete_class, MySimpleInterface) = False


Now if we try and use interface improperly we will get an exception

In [49]:
def some_function(my_interface: MySimpleInterface) -> None:
    print("some_function: my_interface =", str(my_interface)[1:-1])
    print("some_function: calling my_other_interface_function")
    my_interface.my_other_interface_function()

some_function(my_full_concrete_class)
try:
    some_function(my_concrete_class)
except AttributeError:
    print("Exception: Attribute error - my_other_interface is not implemented")

some_function: my_interface = __main__.MyFullImplementedConcreteClass object at 0x000001846D2D1E70
some_function: calling my_other_interface_function
MyFullImplementedConcreteClass: in my_other_interface_function!
some_function: my_interface = __main__.MyConcreteClass object at 0x000001846D2D1AB0
some_function: calling my_other_interface_function
Exception: Attribute error - my_other_interface is not implemented


Virtual base class and metaclass do not appear in the MRO if implicit inheritance used

In [50]:
print("MyConcreteClass.__mro__ =", MyConcreteClass.__mro__)
print("MyFullImplementedConcreteClass.__mro__ =", MyFullImplementedConcreteClass.__mro__)

MyConcreteClass.__mro__ = (<class '__main__.MyConcreteClass'>, <class 'object'>)
MyFullImplementedConcreteClass.__mro__ = (<class '__main__.MyFullImplementedConcreteClass'>, <class 'object'>)


Alternatively you can use the abstract base class `abc` to be more formal

In [51]:
import abc      # abc = abstract base class

class MySimpleABCInterface(metaclass=abc.ABCMeta):

    @classmethod
    def __subclasshook__(cls, subclass):
        print("__subclasshook__: self =", str(cls)[1:-1])
        print("__subclasshook__: subclass =", str(subclass)[1:-1])
        # see if subclass actually implements the interface
        return (hasattr(subclass, 'print_message') and callable(subclass.print_message) and
                hasattr(subclass, 'my_other_interface_function') and callable(subclass.my_other_interface_function) or
                NotImplemented)

    @abc.abstractmethod
    def print_message(self, message: str) -> None:
        raise NotImplementedError

    @abc.abstractmethod
    def my_other_interface_function(self):
        raise NotImplementedError

Instantiating a `MyABCClass` object that implements the interface

In [52]:
class MyABCClass(MySimpleABCInterface):
    def print_message(self, message: str) -> None:
        print("MyABCClass: message =", message)

    def my_other_interface_function(self):
        print("MyABCClass: in my_other_interface_function!")

my_abc_class = MyABCClass()

Instantiating a `NotMyABCClass` object that does not implement all abstract methos in interface

In [53]:
class NotMyABCClass(MySimpleABCInterface):                 # fails to implement all abstract methods
    def print_message(self, message: str) -> None:
        print("NotMyABCClass message:", message)

try:
    my_not_abc_class = NotMyABCClass()
except TypeError:
    print("Trying to instantiate an object that does not implement interface")


Trying to instantiate an object that does not implement interface


Now check the two instances...

In [54]:
print("issubclass(MyABCClass, MySimpleABCInterface) =", issubclass(MyABCClass, MySimpleABCInterface))
# note the following will return True because you have defined methods in interface
print("issubclass(NotMyABCClass, MySimpleABCInterface) =", issubclass(NotMyABCClass, MySimpleABCInterface))

__subclasshook__: self = class '__main__.MySimpleABCInterface'
__subclasshook__: subclass = class '__main__.MyABCClass'
issubclass(MyABCClass, MySimpleABCInterface) = True
__subclasshook__: self = class '__main__.MySimpleABCInterface'
__subclasshook__: subclass = class '__main__.NotMyABCClass'
issubclass(NotMyABCClass, MySimpleABCInterface) = True
